# Вычисление арифметических выражений

В любом языке программирования можно очень просто вычислять арифметические выражения, так как для основных арифметических операций уже есть соответствующие конструкции, такие как + - * /, а практически в каждом языке программирования присутствует библиотека для вычисления математических функций.

Однако "зашитое" в код арифметическое выражение не может быть в последствии изменено! Иногда требуется уметь вычислять произвольную формулу. Классический пример: калькулятор, где пользователь вводит выражение, а программа должна вывести итоговое число.

In [1]:
#include <iostream>
#include <algorithm>
#include <vector>
#include <string>
#include <stack>
#include <sstream>
#include <map>

# Простой подход "В лоб"

Если операции крайне простые и имеют одинаковый приоритет выполнения (например, только + или -), то тогда можно вычислить выражение просто последовательно применяя соответствующие операции к числам.

In [2]:
int n(char s) {
    return static_cast<int>(s-'0'); // превращаем символ числа в число
}

In [3]:
int simple_compute(std::string e) { 
    int operation = 0; // 0 -> плюс, 1 -> минус
    int result = 0; // конечный результат
    int current_num = 0; // текущее считанное число
    for(auto s : e) { // идем по строке
        if(s == '+' or s == '-') { // если операция, то значит мы дочитали предыдущее число и необходимо применить предыдущую операцию
            result = result + (operation == 0 ? current_num : -current_num); // применение операции
            current_num = 0; // обнуляем текущее число
            operation = (s == '+' ? 0 : 1); // обновляем операцию
        } else if(s >= '0' and s <='9') {
            current_num = current_num*10 + n(s); // считываем число
        }
    }
    result = result + (operation == 0 ? current_num : -current_num); // финальное применение последней операции
    return result;
}

In [4]:
std::cout << simple_compute("1+1") << std::endl;

2


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [5]:
std::cout << simple_compute("2+3-7+4-5+4+1-3+9") << std::endl;

8


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [6]:
std::cout << simple_compute("12+78+652-741+23+65-73") << std::endl;

16


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


Данная функция очень проста в реализации и работает очень быстро. Однако такой подход очень ограничен - если мы добавим умножение, то алгоритм уже будет работать некорректно. 1\*2+3\*4 будет вычислен как ((1\*2)+3)\*4, а не как (1\*2)+(3\*4), что очевидно неправильно. Также нет вожможности строить сложные выражения со скобками.

Как можно заметить корнем возникших проблем является неудобная для вычислений запись выражения, так как для того, чтобы понять, как применить операцию нам необходимо посмотреть дальше в строку. 

# Обратная польская запись

Основным способом записи арифметического выражения для его вычисления является обратаня польская запись.

Особенность этой формы записи заключается в том, что вначале идут агрументы и только потом идет операция, которая к ним применяется.

Пример: 2 3 +. Эта запись эквиавалента обычной записи 2+3 и работает она следующим образом: операция + применяется к двум числам идущим за ней.

С помощью обратной польской записи можно записать любое выражение, которое можно записать и в обычной форме.

Примеры:

2+3+4 = 2 3 4 + +

1+2-3 = 1 2 3 - + // вначале вычисляется минус, как 2-3. Потом также применяется +, но уже к 1 и результату минуса.

4+6\*5 = 4 6 5 \* +

6\*5+4 = 6 5 \* 4 + // можно менять слагаемые местами

Обратная польская позволяет записывать сложные выражения со скобками

(2 + 3) \* 7 = 2 3 + 7 \*

(78 + 5 \* 4 ) / (4 + 10) = 78 5 4 \* + 4 10 + /  

Деление будет применятся к результату дву идущих до него плюсов.
первый плюс применится к произведению 5\*4 и 78, второй плюс применится к 4 и 10

Вычисления выражений на обратной польской нотации основываются на использовании стека.

*Стек* - это очень простая структура данных, которая обладает всего двумя операциями: положить элемент на вершину и взять с вершины элемент.

In [7]:
std::stack<int> s; // стек присутствует в стандартной библиотеке
s.push(1); // положить на вершину
s.push(4);
s.push(9);

for(int i = 0; i < 3; i++) {
    std::cout << s.top() << std::endl; // вывести элемент, который сейчас на вершине
    s.pop(); // удалить элемент с вершины
}

9
4
1


Как можно заметить, нам вывелись числа в обратном порядке - последним мы положили на вершину число 9, затем лежала 4 и потом 1.

Основной алгоритм вычисления крайне прост:

* Идем по строке и считываем число или операцию
* Если это число, то кладем его в стек
* Если же это операция, то значит аргументы для этой операции уже лежат в стеке последними (по свойству обратной польской). Достанем необходимое количество аргументов из стека, применим к ним операцию и после этого положим результат обратно на стек.
* После того, как мы обработает таким образом всю строку, на вершине стека будет лежать итоговый результат применения всех операций


In [8]:
class PolishCompute{
public:
    PolishCompute(){}
    ~PolishCompute(){}
    
    long int compute(std::string e) {
        std::stringstream ss(e); // создаем поток, чтобы было удобно из него читать
        std::string current;
        
        while(ss >> current) { // считываем элементы из строки
            if(current == "+") add(); // вызываем соответствующую функцию для каждого оператора
            else if(current == "-") subl();
            else if(current == "*") mult();
            else if(current == "/") div();
            else st.push(current); // и просто кладем на стек, если это число
        }
        
        long int result = std::stoi(st.top());
        st.pop();
        return result;
    }
    
private:
    void add() {
        long int x = std::stoi(st.top()); st.pop();
        long int y = std::stoi(st.top()); st.pop();
        st.push(std::to_string(y + x)); // считаем значение оператора для последних двух чисел на стеке
    }
    
    void subl() {
        long int x = std::stoi(st.top()); st.pop();
        long int y = std::stoi(st.top()); st.pop();
        st.push(std::to_string(y - x));
    }
    
    void mult() {
        long int x = std::stoi(st.top()); st.pop();
        long int y = std::stoi(st.top()); st.pop();
        st.push(std::to_string(y * x));
    }
    
    void div() {
        long int x = std::stoi(st.top()); st.pop();
        long int y = std::stoi(st.top()); st.pop();
        st.push(std::to_string(y / x));
    }
    
    std::stack<std::string> st;
};

In [9]:
PolishCompute pc;
std::cout << pc.compute("2 3 +") << std::endl;

5


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [10]:
std::cout << pc.compute("2 3 4 + +") << std::endl;

9


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [11]:
std::cout << pc.compute("1 2 3 - +") << std::endl;

0


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [12]:
std::cout << pc.compute("4 6 5 * +") << std::endl;

34


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [13]:
std::cout << pc.compute("6 5 * 4 +") << std::endl;

34


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [14]:
std::cout << pc.compute("2 3 + 7 *") << std::endl;

35


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [15]:
std::cout << pc.compute("78 5 4 * + 4 10 + /") << std::endl;

7


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


Все примеры были успешно посчитаны.

Таким образом, мы умеем считать арифметические выражения любой сложности записанные в обратной польской нотации. 

Теперь, если мы будем уметь переводить стандартную запись в обратную польскую, то мы тогда мы сможем успешно вычислять формулы в обычной записи.

# Перевод из стандартной нотации в обратную польскую

Алгоритм будет несколько сложнее предыдущего, однако он все еще очень прост и понятен.

* У нас есть переменная содержащая итоговую строку в обратной польсткой и стек, в котором мы будем хранить операторы
* Нам важно различать различные приоритеты у операторов, так что сделаем дополнительную функцию, которую будет считать приоритет. Пример: + < \* (у умножения приоритет выше), \* < ^ (у возведения в степерь приоритет выше). Сам набор операций и их приоритеты вы можете выставить как угодно. Важно, только чтобы скобки имени самый малый приоритет относительно остальных операций.
* Далее идем по строке
 * Если число, то сразу добавим его к итоговой строке
 * Если открывающая скобка, то кладем ее в стек операторов
 * Если закрывающая скобка, то мы кладем операторы их стека в строку, пока не дойдем до открывающем скобки. Саму открывающую скобку нужно просто убрать из стека
 * Если оператор, то нужно посмотреть на его приоритет и приоритет оператора, который сейчас лежит на стеке:
   * Если у текущего меньше или равен, чем на стеке, то переносим оператор из стека в строку
   * Повторяем предыдущее действие до тех пор, пока не наткнемся на оператор с большим приоритетом или стек не закончится
   * После этого вне зависимости от приоритетов кладем текущий оператор на стек
* После этого кладем все оставшиеся на стеке операторы в строку


In [16]:
class ComputeEngine {
public:
    ComputeEngine(){
        priority["("] = 0;
        priority[")"] = 0;
        priority["+"] = 1; // выставляем приоритет операциям. 
        priority["-"] = 1; // Само значение не важно - главное чтобы их можно было сравнивать
        priority["*"] = 2;
        priority["/"] = 2;
    }
    ~ComputeEngine(){}
    
    int compute(std::string e) {
        return pc.compute(to_polish(e)); // вычисляем значение строки, переведенной в польскую нотацию
    }
    
    std::string to_polish(std::string e) {
        add_spaces(e); // приведем к удобному для четния формату
        
        std::stringstream result; // итоговый (сдела потоком для удобства)
        std::stack<std::string> ops; // операторы
        
        std::stringstream ss(e); // поток для удоства
        std::string current; // текущий
        
        while(ss >> current) { // считываем элементы из ввода
            if(priority.count(current) == 1) { // если для данного символа указан приоритет, то это оператор или скобка
                if(current == "(") {
                    ops.push(current);
                    continue;
                }
                if(current == ")") {
                    while(ops.top() != "(") { // пока не дойдем до открывающей
                        result << ops.top() << " "; // добавляем в строку
                        ops.pop(); // удаляем
                    }
                    ops.pop();
                    continue;
                }
                
                // если оператор
                while(!ops.empty() and priority[current] <= priority[ops.top()]) { // пока больше приоритет
                    result << ops.top() << " "; // добавляем к строке
                    ops.pop(); // удаляем
                }
                ops.push(current);
            } else { // число
                result << current << " ";
            }
        }
        
        while(!ops.empty()) { // оставшиеся операторы
            result << ops.top() << " ";
            ops.pop();
        }
        
        std::string polish_string;
        std::getline(result, polish_string);
        return polish_string;
    }
    
private:
    // для удобного перевода в польскую нотацию, значащие символы
    // но обычно строка вводится без пробелов
    // эта функция добавляет эти пробелы
    void add_spaces(std::string &e) {
        std::stringstream result;
        for(auto s : e) {
            if(priority.count(std::string(1, s)) == 1) result << ' ' << s << ' ';
            else if(s == '(' or s == ')') result << ' ' << s << ' ';
            else result << s;
        }
        std::getline(result, e);
    }
    
    PolishCompute pc;
    std::map<std::string, int> priority;
};

In [17]:
ComputeEngine ce;

std::cout << ce.to_polish("2+3") << std::endl;

2 3 + 


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [18]:
std::cout << ce.to_polish("6*5+4") << std::endl;

6 5 * 4 + 


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [19]:
std::cout << ce.to_polish("4 + 6*5") << std::endl;

4 6 5 * + 


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [20]:
std::cout << ce.to_polish("(2 + 3) * 7") << std::endl;

2 3 + 7 * 


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [21]:
std::cout << ce.to_polish("(78 + 5 * 4 ) / (4 + 10)") << std::endl;

78 5 4 * + 4 10 + / 


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [22]:
std::cout << ce.to_polish("275 * ( (45+3) / (78-12*14) ) / (17+45-23*75)") << std::endl;

275 45 3 + 78 12 14 * - / * 17 45 + 23 75 * - / 


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


**Проверим корректность вычислений выражений**

In [23]:
std::cout << ce.compute("2+3") << std::endl;

5


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [24]:
std::cout << ce.compute("6*5+4") << std::endl;

34


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [25]:
std::cout << ce.compute("4 + 6*5") << std::endl;

34


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [26]:
std::cout << ce.compute("(2 + 3) * 7") << std::endl;

35


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [27]:
std::cout << ce.compute("(78 + 5 * 4 ) / (4 + 10)") << std::endl;

7


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0


In [28]:
std::cout << ce.compute("2775 * ( 9654*(45+3) / (78-12*14) ) / (17+45-23*75)") << std::endl;

8590


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f563b3b46e0
